In [2]:
import csv
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from IPython.display import Image
import seaborn as sns


import time
from selenium import webdriver

C:\Users\Arina\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
# Scrape the job listings with Selenium
# driver = webdriver.Chrome('C:/Users/Arina/Chromedriver/chromedriver')  

In [3]:
# pandas display settings

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 3)

In [4]:
# Read in raw email data
filepath = '..\data_local\jobhunt_zapier_022818.csv'
df = pd.read_csv(filepath)
df.head()

,email_from,email_subject,email_content,date
0,Indeed Job Alert,You have new recommended jobs: Data Scientist ...,Recommended Jobs for You\r\n\r\nJobs 1 to 20 o...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)"
1,Idealist,22 new listings match your search on Idealist,"\n\n\nHi Arina,\n\n\n\nWrite it Down, Make it ...",26 Feb 2018 15:53:22 -0500
2,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Mon, 26 Feb 2018 18:18:35 -0600 (CST)"
3,LinkedIn Job Alerts,Arina: 667 new jobs for 'data science',LinkedIn | 667 new jobs matching your search f...,"Tue, 27 Feb 2018 01:00:46 +0000 (UTC)"
4,We Work Remotely,"Listings for Tue, Feb 27",Your daily We Work Remotely job listings\r\n\r...,"Tue, 27 Feb 2018 04:28:27 +0000 (UTC)"


In [5]:
# testing
df.email_from[0].strip().lower()

'indeed job alert'

In [6]:
if df.email_from[0].strip().lower() in ('rodolphe dutel'):
    print('test')

In [7]:
from bs4 import BeautifulSoup

In [8]:
# Returns an array of cleaned links
def parse_email_content(email_from, email_subject, email_content, email_date):
    links_list_clean = []
        
    # different emails have different format
    # process them per email sender
    if email_from.strip().lower() in ('rodolphe dutel'):
        
        # clean up the email content, get the chunk that contains job postings links
        email_content = re.sub(r'\r\n', r'', email_content)
        email_content = re.sub(r'unsubscribe.*', r'', email_content, flags=re.IGNORECASE)
        email_content = re.sub(r'.*-Rodolphe', r'', email_content, flags=re.IGNORECASE)
        
        # extract a list of job post links in the email
        p =r"(?<=---).{1,100} \( ?https://\S* \)-*"
        links_list = re.findall(p, email_content)
        
        # clean the extracted list of job post links
        for linky in links_list:
            linky_clean = re.search(r'(?<=\() ?https?://.*(?=\))', linky, re.IGNORECASE).group().strip()
            # check for dups
            if linky_clean not in set(links_list_clean):
                links_list_clean.append(linky_clean)

        # process the cleaned links to get the job posts data
        for linky_url in links_list_clean:
            # go to the job post link
            driver.get(linky_url)
            
            # get job title
            selector = '//h2[@class="page-title"]'
            elems = driver.find_elements_by_xpath(selector)
            job_title = elems[0].text if len(elems) > 0 else ''
            job_titles.append(job_title)
            
            # get company name
            selector = '//ul[@class="job-listing-meta meta"]/li[@class="job-company"]/a'
            elems = driver.find_elements_by_xpath(selector)
            company_name = elems[0].text if len(elems) > 0 else ''
            company_names.append(company_name)
            
            # get location
            selector = '//ul[@class="job-listing-meta meta"]/li[@class="location"]'
            elems = driver.find_elements_by_xpath(selector)
            location = elems[0].text if len(elems) > 0 else ''
            locations.append(location)
            
            # get date posted
            selector = '//ul[@class="job-listing-meta meta"]/li[@class="date-posted"]'
            elems = driver.find_elements_by_xpath(selector)
            date_posted = elems[0].text if len(elems) > 0 else ''
            date_posteds.append(date_posted)
            
            # get job posting
            selector = "//div[contains(concat(' ', @class, ' '), ' job-overview ')]"
            elems = driver.find_elements_by_xpath(selector)
            job_posting = elems[0].text if len(elems) > 0 else ''
            job_postings.append(job_posting)
            
            # get the actual job post link (instead of email referral link)
            job_post_links.append(driver.current_url)
        
            # get company description
            selector = "//div[contains(concat(' ', @class, ' '), ' job-company-about ')]"
            elems = driver.find_elements_by_xpath(selector)
            company_descr = elems[0].text if len(elems) > 0 else ''
            company_descrs.append(company_descr)
            
    
    elif email_from.strip().lower() in ('indeed job alert', 'indeed'):
        # clean up the email content, get the chunk that contains job postings links
        email_content = re.sub(r'\r\n', r' ', email_content)
        email_content = re.sub(r'Cancel this Recommended Job Alert:.*', r'', email_content, flags=re.IGNORECASE)
        
        # extract a list of job post links in the email
        p =r"https://www.indeed.com/rc/\S* "
        links_list = re.findall(p, email_content)
        
        # clean the extracted list of job post links
        for linky in links_list:
            linky_clean = linky.strip()
            # check for dups
            if linky_clean not in set(links_list_clean):
                links_list_clean.append(linky_clean)

        # process the cleaned links to get the job posts data
        for linky_url in links_list_clean:
            # go to the job post link
            driver.get(linky_url)
            
            # get job title
            selector = '//table[@id="job-content"]//b[@class="jobtitle"]'
            elems = driver.find_elements_by_xpath(selector)
            job_title = elems[0].text if len(elems) > 0 else ''
            job_titles.append(job_title)
            
            # get company name
            selector = '//table[@id="job-content"]//span[@class="company"]'
            elems = driver.find_elements_by_xpath(selector)
            company_name = elems[0].text if len(elems) > 0 else ''
            company_names.append(company_name)
            
            # get location
            selector = '//table[@id="job-content"]//span[@class="location"]'
            elems = driver.find_elements_by_xpath(selector)
            location = elems[0].text if len(elems) > 0 else ''
            locations.append(location)
            
            # get date posted
            selector = '//span[@class="date"]'
            elems = driver.find_elements_by_xpath(selector)
            date_posted = elems[0].text if len(elems) > 0 else ''
            date_posteds.append(date_posted)
            
            # get job posting
            selector = '//span[@id="job_summary"]'
            elems = driver.find_elements_by_xpath(selector)
            job_posting = elems[0].text if len(elems) > 0 else ''
            job_postings.append(job_posting)
            
            # get the actual job post link (instead of email referral link)
            job_post_links.append(driver.current_url)
            
            # get company description
            # go to the company profile page
            click_selector = '//div[@class= "cmp_info"]//div[@class= "cmp_title"]/a'
            elems = driver.find_elements_by_xpath(click_selector)
            if len(elems) > 0:
                elems[0].click()
                # get short company descr
                selector = '//span[@id="cmp-short-description"]'
                elems = driver.find_elements_by_xpath(selector)
                company_descr1 = elems[0].text if len(elems) > 0 else ''
                # get more company descr - with BeautifulSoup
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                soupy = soup.findAll('span',attrs={'id':'cmp-more-description'})
                if soupy:
                    company_descr2 = str(soupy[0])
                    company_descr2 = re.sub(r'<a ?.* id="cmp-less-link">less</a>', r'', company_descr2, flags=re.IGNORECASE)
                    company_descr2 = re.sub(r'<span ?.* id="cmp-more-description">', r'', company_descr2, flags=re.IGNORECASE)
                    company_descr2 = re.sub(r'</span>', r'', company_descr2, flags=re.IGNORECASE)
                    company_descr2 = re.sub(r'(<br>)|(<br/>)', r'\r\n', company_descr2, flags=re.IGNORECASE)
                    company_descr2 = re.sub(r'&amp;', r'and', company_descr2, flags=re.IGNORECASE)
                else:
                    company_descr2 = ''
            
                company_descr = company_descr1 + ' ' + company_descr2
            else:
                company_descr = ''
            
            company_descrs.append(company_descr)
            
    
    elif email_from.strip().lower() in ('linkedin job alerts'):
        # clean up the email content, get the chunk that contains job postings links
        # -- LinkedIn job post emails are clean enough as is!
        
        # extract a list of job post links in the email
        p =r"https://www.linkedin.com/comm/jobs/view/\S*"
        links_list = re.findall(p, email_content)
        
        # clean the extracted list of job post links
        for linky in links_list:
            linky_clean = linky.strip()
            # check for dups
            if linky_clean not in set(links_list_clean):
                links_list_clean.append(linky_clean)

        # process the cleaned links to get the job posts data
        for linky_url in links_list_clean:
            # go to the job post link
            driver.get(linky_url)
            
            # get job title
            selector = '//section[@id="top-card"]//h1[@class="title"]'
            elems = driver.find_elements_by_xpath(selector)
            job_title = elems[0].text if len(elems) > 0 else ''
            job_titles.append(job_title)
            
            # get company name
            selector = '//section[@id="top-card"]//span[@class="company"]'
            elems = driver.find_elements_by_xpath(selector)
            company_name = elems[0].text if len(elems) > 0 else ''
            company_names.append(company_name)
            
            # get location
            selector = '//section[@id="top-card"]//h3[@class="location"]/span/span'
            elems = driver.find_elements_by_xpath(selector)
            location = elems[0].text if len(elems) > 0 else ''
            locations.append(location)
            
            # get date posted
            selector = '//section[@id="top-card"]//ul[@class="posting-info"]/li[@class="posted"]/span'
            elems = driver.find_elements_by_xpath(selector)
            date_posted = elems[0].text if len(elems) > 0 else ''
            date_posteds.append(date_posted)
            
            # get job posting
            selector = '//section[@id="summary-detail"]//div[@class="description-section"]'
            elems = driver.find_elements_by_xpath(selector)
            job_posting = elems[0].text if len(elems) > 0 else ''
            job_postings.append(job_posting)
            
            # get the actual job post link (instead of email referral link)
            job_post_links.append(driver.current_url)
            
            # get company description
            selector = '//section[@id="company-lcp"]//div[@class="description-container"]'
            elems = driver.find_elements_by_xpath(selector)
            company_descr = elems[0].text if len(elems) > 0 else ''
            company_descrs.append(company_descr)
            
            
            
            
            
            
            
    # For each job link, record email info
    for linky_url in links_list_clean:
        email_froms.append(email_from)
        email_subjects.append(email_subject)
        email_contents.append(email_content)
        email_dates.append(email_date)

        
    

In [9]:
# testing
job_titles = []
company_names = []
locations = []
date_posteds = []
job_postings = []
company_descrs = []
job_post_links = []

email_froms = []
email_subjects = []
email_contents = []
email_dates = []

# Scrape the job listings with Selenium
driver = webdriver.Chrome('C:/Users/Arina/Chromedriver/chromedriver')  

# process all emails in the dataframe
for i in range(len(df)):
    parse_email_content(df.email_from[i], df.email_subject[i], df.email_content[i], df.date[i])

# close the Selenium browser
driver.close()

print(job_titles)

['Data Scientist', 'Machine Learning Educator / Research Engineer', 'Systems Software Engineer - Neuroscience', 'Data Scientist - Intermediate', 'Data Analyst', 'Data Science Intern', 'Data Scientist Job', 'Data Scientist', 'PEM Membrane Scientist', 'Associate Analyst-Data Scientist', 'Data & Applied Scientist', 'Mathematical Statistician', 'Senior Data Scientist (Open to Remote)', 'Data Analyst / Data Scientist', 'Material Analyst - Pre-Review', '', 'Tableau Developer', 'Technical Project Manager', 'Principal Data Scientist', 'Data Scientist', 'Data Science Analytics Leader', 'Data Science Statistician', 'Data Science Associate', '', 'Principal Data Scientist - Predictive Analytics, Machine Learning', 'Data Scientist', 'Junior Imaging Sensor Data Scientist', 'Data Scientist', 'Data Scientist', 'IBM Data Scientist: Apprentice', 'Data Scientist (18016)', 'Data Science Intern, Summer 2018', 'Director of Data Science', 'Remote Freelance Engineering / Data Mentor', 'Data Scientist', 'Data 

In [10]:
print(company_descrs)

['', 'Bloomberg is a company dedicated to helping solve complex challenges through insight and information. Our strength—quickly and accurately delivering data, news and analytics through technology—is at the core of everything we do. With over 19,000 employees in 192 locations, we give influential decision makers in business, finance and government a competitive edge by connecting them to a dynamic network of news, people and ideas. To do so, we need constant energy and innovation—which is where you come in.\r\n\r\nAt Bloomberg, you will have the opportunity to take risks and be part of an organization that is entering new markets, launching new ventures and pushing the boundaries. Our ever-expanding technology, data, news and media services afford employees the opportunity to expand skills and connect with smart, driven colleagues from a diversity of backgrounds and ideas.\r\n\r\nWe’re looking for dynamic, multi-talented people who have a desire to thrive in a forward-thinking cultur

In [11]:
df_clean = pd.DataFrame(
    {
     'email_from': email_froms,
     'email_subject': email_subjects,
     'email_content': email_contents,
     'email_date': email_dates,
     'job_title': job_titles,
     'company_name': company_names,
     'location': locations,
     'date_posted': date_posteds,
     'job_posting': job_postings,
     'company_descr': company_descrs,
     'job_post_link': job_post_links
    })
df_clean

,company_descr,company_name,date_posted,email_content,email_date,email_from,email_subject,job_post_link,job_posting,job_title,location
0,,Geophy,3 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=1fbba60eb2c9...,GeoPhy is a company that offers independent pr...,Data Scientist,"New York, NY"
1,Bloomberg is a company dedicated to helping so...,Bloomberg,4 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=a1e3470c4783...,Job Requisition Number:65487\n\nAt Bloomberg's...,Machine Learning Educator / Research Engineer,"New York, NY"
2,,Simons Foundation,5 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=d4a1c8ed325a...,The Simons Foundation is beginning a new compu...,Systems Software Engineer - Neuroscience,"New York, NY 10010"
3,"Just because you match a job on paper, doesn't...",KellyMitchell,4 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=4dbf56a8c979...,KellyMitchell matches the best IT and business...,Data Scientist - Intermediate,"Chesterfield, MO"
4,"Headquartered in Woonsocket, Rhode Island, CVS...",CVS Health,5 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=83793cb590f2...,The Systems Data Analyst is responsible for pe...,Data Analyst,"New York, NY"
5,,Lazard Ltd.,4 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=86e08c21ad96...,Lazard\nSummer Data Science Intern\n\nJob Desc...,Data Science Intern,"New York, NY"
6,,SAIC,4 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=deb29dc391c3...,Data Scientist (Job Number:434442)\n\nDescript...,Data Scientist Job,"Huntsville, AL"
7,Alliance Data (NYSE: ADS)is a leading provider...,AllianceData,6 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=f47a67c0048f...,As a Data Scientist in our Decision Sciences R...,Data Scientist,"Chicago, IL"
8,"Ford Motor Company, a global automotive indust...",Ford Motor Company,5 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=4051aa3e9db0...,Job Description\nFord Research and Advanced En...,PEM Membrane Scientist,"Dearborn, MI 48121"
9,"At Abercrombie & Fitch, quality is in our root...",Abercrombie & Fitch,11 days ago,Recommended Jobs for You Jobs 1 to 20 of 20 r...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)",Indeed Job Alert,You have new recommended jobs: Data Scientist ...,https://www.indeed.com/viewjob?jk=e03c46acb259...,Associate Analyst-Data Scientist Requirements\...,Associate Analyst-Data Scientist,"Columbus, OH"


In [12]:
# Pickle the cleaned job posts dataset
df_clean.to_pickle('../data_local/job_posts_parsed_022818.pkl')

In [13]:
len(df_clean)

128

In [14]:
len(df)

12

In [15]:
df

,email_from,email_subject,email_content,date
0,Indeed Job Alert,You have new recommended jobs: Data Scientist ...,Recommended Jobs for You\r\n\r\nJobs 1 to 20 o...,"Mon, 26 Feb 2018 13:47:08 -0600 (CST)"
1,Idealist,22 new listings match your search on Idealist,"\n\n\nHi Arina,\n\n\n\nWrite it Down, Make it ...",26 Feb 2018 15:53:22 -0500
2,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Mon, 26 Feb 2018 18:18:35 -0600 (CST)"
3,LinkedIn Job Alerts,Arina: 667 new jobs for 'data science',LinkedIn | 667 new jobs matching your search f...,"Tue, 27 Feb 2018 01:00:46 +0000 (UTC)"
4,We Work Remotely,"Listings for Tue, Feb 27",Your daily We Work Remotely job listings\r\n\r...,"Tue, 27 Feb 2018 04:28:27 +0000 (UTC)"
5,Indeed Job Alert,You have new recommended jobs: Data Scientist ...,Recommended Jobs for You\r\n\r\nJobs 1 to 20 o...,"Tue, 27 Feb 2018 13:27:48 -0600 (CST)"
6,Dice Job Alert,Dice Job Alert - Data science,The career hub for tech\n\nDice Job Alert\n\nH...,"Tue, 27 Feb 2018 13:30:49 -0600 (CST)"
7,Idealist,26 new listings match your search on Idealist,"\n\n\nHi Arina,\n\n\n\nWrite it Down, Make it ...",27 Feb 2018 16:05:05 -0500
8,Indeed,30+ new data science jobs,Indeed Job Alert\r\ndata science jobs\r\n\r\nJ...,"Tue, 27 Feb 2018 18:01:23 -0600 (CST)"
9,LinkedIn Job Alerts,Arina: 915 new jobs for 'data science',LinkedIn | 915 new jobs matching your search f...,"Wed, 28 Feb 2018 01:24:21 +0000 (UTC)"


In [16]:
df_clean.email_from.value_counts()

Indeed                 66
Indeed Job Alert       40
LinkedIn Job Alerts    22
Name: email_from, dtype: int64